# Statistiques descriptives sur le profil 2023


In [1]:
# import des packages
from donnees_IDFM import donnees_archives, donnees_validation_23, donnees_profils_23, donnees_stations_toutes
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
import plotly.express as px


In [2]:
# importation des deux bases de données 2023
validations_23=donnees_validation_23()
profils_23=donnees_profils_23()
ref_stations =donnees_stations_toutes()

In [33]:
profils_23.head()

,CODE_STIF_TRNS,CODE_STIF_RES,CODE_STIF_ARRET,LIBELLE_ARRET,lda,CAT_JOUR,TRNC_HORR_60,pourc_validations
84132,800,852,62,BELLEVUE,69528,SAVS,6H-7H,1.84
84133,800,852,657,PLAISIR GRIGNON,64199,DIJFP,17H-18H,8.30
84134,800,852,657,PLAISIR GRIGNON,64199,DIJFP,5H-6H,0.31
84135,800,852,657,PLAISIR GRIGNON,64199,DIJFP,8H-9H,4.92
84136,800,852,657,PLAISIR GRIGNON,64199,JOHV,14H-15H,3.02


Les **CAT_JOUR** sont les suivantes : 
    - **JOVS** : Jour ouvré en vacances scolaires
    - **JOHV** : Jour ouvré hors vacances 
    - **SAHV** : Samedi hors vacances
    - **SAVS** : Samedi en vacances scolaires
    - **DIJFP** : Dimanche et jours fériés

Pour une même station, on a donc normalement 24*5 = 120 lignes, mais on remarque que certaines lignes manquent (exemple sur la station Gallieni qui compte 119 lignes, avec la tranche horaire 3h-4h en SAVS manquante). 

In [55]:
_0 = profils_23[profils_23['pourc_validations'] == 0.00]
len(_0)

3001

Ces valeurs manquantes ne sont pas dûes à une absence de validation : on a bien des lignes qui apparaissent à des tranches horaires vides.

In [60]:
cat_jour = profils_23.groupby('lda')['CAT_JOUR'].mean()
cat_jour


TypeError: agg function failed [how->mean,dtype->object]

In [7]:
# début script Emma 

# nombre et liste des valeurs distinctes par variable

for colonne in profils_23.columns:
    nombre_distinct = profils_23[colonne].nunique()
    
    if nombre_distinct <= 15: #borne de 15 choisie après une première observation du nombre de variables différentes pour chaque variable et pour avoir des groupes cohérents
        valeurs_distinctes = profils_23[colonne].unique()
        
        print(f"Nombre de valeurs distinctes dans '{colonne}' est :", nombre_distinct)
        print(f"Liste des valeurs distinctes dans '{colonne}' est :", valeurs_distinctes)
        print()

    else :
        print(f"Nombre de valeurs distinctes dans '{colonne}' est :", nombre_distinct)
        print()

Nombre de valeurs distinctes dans 'CODE_STIF_TRNS' est : 3
Liste des valeurs distinctes dans 'CODE_STIF_TRNS' est : [100 800 810]

Nombre de valeurs distinctes dans 'CODE_STIF_RES' est : 13
Liste des valeurs distinctes dans 'CODE_STIF_RES' est : ['110' 'ND' '800' '803' '802' '804' '805' '850' '851' '852' '854' '801'
 '853']

Nombre de valeurs distinctes dans 'CODE_STIF_ARRET' est : 756

Nombre de valeurs distinctes dans 'LIBELLE_ARRET' est : 747

Nombre de valeurs distinctes dans 'lda' est : 726

Nombre de valeurs distinctes dans 'CAT_JOUR' est : 5
Liste des valeurs distinctes dans 'CAT_JOUR' est : ['JOVS' 'SAHV' 'SAVS' 'DIJFP' 'JOHV']

Nombre de valeurs distinctes dans 'TRNC_HORR_60' est : 25

Nombre de valeurs distinctes dans 'pourc_validations' est : 2452



On voit qu'on va avoir à peu près les mêmes soucis que sur la base précédente : 
- différents types de réseaux : ici du ND qui apparait > pas très grave, car dans tous les cas on sélectionne que les 5 lignes de RER
- les 3 variables qui permettent d'avoir le nom de l'arrêt n'ont pas le nême nombre de valeurs uniques : il va falloir travailler pour distinguer les problèmes
- ici on n'a pas la variable année / mois car c'est par type de jour
- les variables explicatives d'intéret que sont CAT_JOUR et TRNC_HORR_60 ont peu de valeurs distinctes : on va pouvoir bien les comprendre et travailler dessus

On n'a pas le même nombre de lignes que sur la base précédente : pourquoi ? quel niveau d'agrégation différent ?